In [1]:
library(tidyverse)
source('../0_support-files/theme_CRP-MISC.R')



expGroupPalette <- c("Control_Non-inflammatory" ='#FBE77C',
                            "Acute COVID-19 Mild/ASX" = '#F0484E',
                            "Acute COVID-19 Moderate/Severe" = '#F0484E',
                            "Acute MIS-C Moderate/Severe" = '#5CB2EB')


── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



---
## Get Mito Count for MIS-C

In [2]:
# path = "/workdir/cjl332/cfdna/alignment/output/MISC/sample_output/"

# get_mito <- function(x,path){
#     output <- system(paste0("cut -f 1,3 ",path,x,"/",x,".idxstats"), intern=T)
#     output <- data.frame(output) %>%
#             mutate(chr = gsub("\\t.*","",output), count =  gsub(".*\\t","",output)) %>% 
#             select(!output)
#     colnames(output)[2] <- x
#     return(output)
# }

# ids <- list.files(path)
# ids <- read.delim("/workdir/cjl332/cfdna/alignment/prep_tables/sequencing_prep.usingMISC.tsv")
# ids <- ids$sample

# mito_counts <- lapply(ids,get_mito,path)
# #combine
# mito_df <- Reduce(function(dtf1, dtf2) merge(dtf1, dtf2, by = "chr"),
#         mito_counts)

# # tidy
# mito_df <- data.frame(mito_df[-1,])
# rownames(mito_df) <- mito_df$chr
# mito_df$chr <- NULL

# # transpose
# mito_df <- data.frame(t(mito_df))

# mito_df[] <- lapply(mito_df,as.numeric)

# mito_df$total_reads <- rowSums(mito_df)

# mito_df$Sequence_ID = gsub("\\_[^\\_]*\\_[^\\_]*$","",gsub("^[^\\_]*\\_[^\\_]*\\_[^\\_]*\\_[^\\_]*\\_","",rownames(mito_df)))

# # head(mito_df)

# cfdna_id_key <- read.delim("../make_datasets/cfdna_id_key.tsv") %>%
#     mutate(Seq_ID = paste0("X",Seq_ID))

# mito_df <- merge(mito_df,cfdna_id_key, by.x=0, by.y="Seq_ID")
# rownames(mito_df) <- mito_df$cfdna_sample_id
# mito_df <- mito_df %>% select(!c("Sequence_ID","cfdna_sample_id","Row.names"))


# write.table(mito_df,"../1_sample-data/cfdna_chr_cnts.tsv",sep="\t",quote=F,row.names=T)

mito_df = read.delim("../1_sample-data/cfdna_chr_cnts.tsv")

In [3]:
meta = read.csv("../1_sample-data/STable8_cfdna-samples.csv") %>% 
    rename(scaled_cfDNA = cfDNA_concentration) %>%
    mutate(severity = ifelse(severity == 0 | severity == 1,"Mild/ASX",
                             ifelse(severity == 2 | severity == 3,"Moderate/Severe",""))) %>%
    mutate(Diagnosis = ifelse(Diagnosis != "Control_Non-inflammatory", paste0("Acute ",Diagnosis," ",severity),Diagnosis)) %>%
    rename(Sample_ID = cfdna_sample_id)%>% 
    mutate(expGroup_plotting = Diagnosis)

head(meta)

# MERGE
all_df <- merge(mito_df,meta,by.x=0,by.y="Sample_ID")

all_df$expGroup_plotting <- factor(all_df$expGroup_plotting,levels = names(expGroupPalette))

nrow(meta)
nrow(mito_df)
nrow(all_df)

# ADD SEVERITY
# all_df <- all_df %>% mutate(severity_v=ifelse(Diagnosis == "Control_Non-inflammatory","control",
#                                               ifelse(Diagnosis == "Control_Inflammatory","control",
#                                                      ifelse(severity %in% c("2","3"), "moderate/severe",
#                                                             ifelse(severity %in% c("0","1"), "asympomatic/mild","other"))))) %>% 
#                     mutate(Diagnosis_v = ifelse(Diagnosis == "Control_Non-inflammatory","pControl",
#                                                 ifelse(Diagnosis == "Control_Inflammatory","pControl_Inflammatory",
#                                                 ifelse(Diagnosis == "COVID-19","pCOVID-19",Diagnosis))))



# all_df$Diagnosis_v <- factor(all_df$Diagnosis_v, levels = c("pControl_Inflammatory","pControl","pCOVID-19","MIS-C"))
# all_df$severity_v <- factor(all_df$severity_v, levels = c("control","asympomatic/mild","moderate/severe"))

# all_df$x_label <- paste0(all_df$Diagnosis_v," ", all_df$severity_v)
# all_df$x_label <- factor(all_df$x_label,levels= c("pControl control","pCOVID-19 asympomatic/mild","pCOVID-19 moderate/severe","MIS-C moderate/severe"))

,PTID,Sample_ID,Diagnosis,severity,timepoint,ivig,days_ivig_before_samp,ivig_rel_samp,sequencing_depth,mapping_efficiency,⋯,pancreas,progenitor,skin,spleen,tcell,other,ALT.max,Creatinine.max,CRP.max,expGroup_plotting
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<chr>
1,Patient38,cfdna_sample_id_24,Acute COVID-19 Moderate/Severe,Moderate/Severe,acute,0,NA,NA,0.966009,0.78,⋯,0.00000000,0.077464020,0.00000000,0.12904566,0.00000000,0,24,0.52,7.4,Acute COVID-19 Moderate/Severe
2,Patient40,cfdna_sample_id_25,Acute MIS-C Moderate/Severe,Moderate/Severe,acute,1,-2,before,0.198854,0.75,⋯,0.01019488,0.003604042,0.00000000,0.00000000,0.00000000,0,64,1.49,15.5,Acute MIS-C Moderate/Severe
3,Patient43,cfdna_sample_id_26,Acute MIS-C Moderate/Severe,Moderate/Severe,acute,1,-1,before,0.538687,0.76,⋯,0.00000000,0.082368089,0.01111545,0.07828432,0.03048947,0,104,1.41,17.9,Acute MIS-C Moderate/Severe
4,Patient44,cfdna_sample_id_27,Acute COVID-19 Mild/ASX,Mild/ASX,acute,0,NA,NA,0.939336,0.78,⋯,0.00264551,0.042459565,0.00000000,0.03866812,0.00000000,0,77,0.59,15.1,Acute COVID-19 Mild/ASX
5,Patient45,cfdna_sample_id_28,Acute MIS-C Moderate/Severe,Moderate/Severe,acute,1,NA,NA,0.533339,0.74,⋯,0.00000000,0.058378850,0.00000000,0.23931450,0.00000000,0,56,1.91,30.2,Acute MIS-C Moderate/Severe
6,Patient46,cfdna_sample_id_29,Acute MIS-C Moderate/Severe,Moderate/Severe,acute,1,-3,before,0.379538,0.77,⋯,0.01899220,0.012543671,0.00000000,0.04158856,0.01250763,0,96,0.60,13.2,Acute MIS-C Moderate/Severe


[1] 65

[1] 65

[1] 63

In [4]:
maxY = 0.003


pdf(file="./plots/panelD.pdf",
                 width=2.35,height=1.5, paper="special", bg="white",
                 fonts="Helvetica", colormodel = "srgb", pointsize=6, useDingbats = FALSE)



all_df %>% 
# ggplot(aes(x=expGroup_plotting,y=chrM,fill=expGroup_plotting))+
ggplot(aes(x=expGroup_plotting,y=(chrM/total_reads)*scaled_cfDNA,fill=expGroup_plotting))+
geom_boxplot(outlier.shape=NA,size = 0.2, color= "black")+
geom_point(position = position_jitterdodge(jitter.width = 0.5,jitter.height = 0),size = 0.2)+
labs(y="Mitochondrial abundance",title="Abundance of mitochondrial DNA") +
    theme_prevail()+
    theme(plot.title = element_blank(),
            text = element_text(size = 8),
         legend.position = "none",
         axis.title.x = element_blank()) + 
scale_fill_manual(values = expGroupPalette) +
coord_cartesian(ylim = c(0,maxY))


dev.off()
# ggsave("./plots/chrM.conc.plt.pdf",width = 2.5, height = 1.5)

png 
  2

In [21]:
library(ggpubr)
library(rstatix)

GROUP = "Control_Non-inflammatory"

stat.test <- data.frame(all_df) %>% 
    wilcox_test( chrM ~ expGroup_plotting, paired = FALSE) %>% 
    adjust_pvalue(method = "BH") %>% 
    add_significance("p.adj") %>% 
    add_xy_position(x = "expGroup_plotting") #%>% 
    # filter(group1 %in% c(GROUP) | group2 %in% c(GROUP)  )

stat.test

.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif,y.position,groups,xmin,xmax
<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<named list>,<dbl>,<dbl>
chrM,Control_Non-inflammatory,Acute COVID-19 Mild/ASX,3,10,26,0.077000,0.092400,ns,104151.4,"Control_Non-inflammatory, Acute COVID-19 Mild/ASX",1,2
chrM,Control_Non-inflammatory,Acute COVID-19 Moderate/Severe,3,11,33,0.005000,0.010000,**,116285.2,"Control_Non-inflammatory , Acute COVID-19 Moderate/Severe",1,3
chrM,Control_Non-inflammatory,Acute MIS-C Moderate/Severe,3,41,123,0.000151,0.000906,***,128418.9,"Control_Non-inflammatory , Acute MIS-C Moderate/Severe",1,4
chrM,Acute COVID-19 Mild/ASX,Acute COVID-19 Moderate/Severe,10,11,87,0.024000,0.036000,*,140552.6,"Acute COVID-19 Mild/ASX , Acute COVID-19 Moderate/Severe",2,3
chrM,Acute COVID-19 Mild/ASX,Acute MIS-C Moderate/Severe,10,41,340,0.000822,0.002466,**,152686.4,"Acute COVID-19 Mild/ASX , Acute MIS-C Moderate/Severe",2,4
chrM,Acute COVID-19 Moderate/Severe,Acute MIS-C Moderate/Severe,11,41,254,0.536000,0.536000,ns,164820.1,"Acute COVID-19 Moderate/Severe, Acute MIS-C Moderate/Severe",3,4
